# Smoker Detector AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to detect cigarate smoker using a smart AI camera. In this project, we will expolre deep learning to identify a smoker in a video stream. To do this, we first build our base model. We will use a pretrained imageNet model and replace ImageNet last layer with a logistic classifier ( binary classifier ) as our overall application architecture.

**Goal** : Design and Build a Deep Learning Model to identify a cigarret smoker in a video stream.

### Base Model

The project (base model) is broken down into multiple steps:

* Load and preprocess the videos dataset if required
* Build a logistic classifier
* Train the classifier on our video dataset
* Use the trained classifier to predict

![Deep Learning](deep_learn_b.jpg)


**TO DO**

1. What input data is required? How you can represent input data i.e videos for imageNet model to consume?

2. What should be an ideal video resolution?

3. What is the outout?

4. What Architecture? ( with respect to the above diagram? )

5. What loss function to use for this model?


First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [0]:
# Imports here
import torch
from torchvision import datasets, transforms
import torchvision.models as models
from torch import nn
from torch import optim
from PIL import Image
import numpy as np
import json


%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). You'll also need to make sure the input data is resized to 224x224??? pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.
 

In [0]:
data_dir = 'videos'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [0]:
# TODO: Define your transforms for the training, validation, and testing sets



### Label mapping



# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier ( Logistic Classifier ), using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters


When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.


In [0]:
# TODO: Build and train your network




In [0]:
# Training the model on the training dataset


## Testing your network

It's good practice to test your trained network on test data, images/videos the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new videos. Run the test videos through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [0]:
# TODO: Do validation on the test set



## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. 

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [0]:
# TODO: Save the checkpoint 

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [0]:
#Loading checkpoint



# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class. Write a function called `predict` that takes an video and a model, 

First you'll need to handle processing the input image such that it can be used in your network. 

## Video Preprocessing



In [0]:
# TODO: Process a PIL image for use in a PyTorch model


